# Combinar conjuntos de datos: Concat

Algunos de los estudios de datos más interesantes proceden de la combinación de distintas fuentes de datos.
Estas operaciones pueden implicar cualquier cosa, desde una concatenación muy directa de dos conjuntos de datos diferentes, hasta uniones y fusiones más complicadas al estilo de las bases de datos que manejan correctamente cualquier solapamiento entre los conjuntos de datos.
``Series`` y ``DataFrame`` se construyen con este tipo de operaciones en mente, y Pandas incluye funciones y métodos que hacen que este tipo de manipulación de datos sea rápida y sencilla.

Aquí echaremos un vistazo a la simple concatenación de ``Series`` y ``DataFrame``s con la función ``pd.concat``; más tarde nos sumergiremos en fusiones y uniones en memoria más sofisticadas implementadas en Pandas.

Comenzamos con las importaciones estándar:

In [ ]:
import pandas as pd
import numpy as np

Por conveniencia, definiremos esta función que crea un ``DataFrame`` de una forma particular que será útil más adelante:

In [ ]:
def make_df(cols:str, ind:list[int]) -> pd.DataFrame:
    """Crear rápidamente un DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# ejemplo DataFrame
make_df('ABC', range(3))

Además, crearemos una clase rápida que nos permita mostrar múltiples ``DataFrame`` uno al lado del otro. El código hace uso del método especial ``_repr_html_``, que IPython utiliza para implementar su visualización de objetos enriquecidos:

In [ ]:
class Display(object):
    """Mostrar la representación HTML de varios objetos"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    
    def __init__(self, *args, context=None):
        # Si no se pasa un contexto, se usa el entorno local por defecto
        if context is None:
            context = globals()
        
        # Convertir los nombres de variables a objetos reales si son cadenas
        self.args = [eval(a, context) if isinstance(a, str) else a for a in args]
        self.arg_names = [a if isinstance(a, str) else repr(a) for a in args]
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(name, obj._repr_html_())
                         for name, obj in zip(self.arg_names, self.args))
    
    def __repr__(self):
        return '\n\n'.join(name + '\n' + repr(obj)
                           for name, obj in zip(self.arg_names, self.args))

Su utilidad quedará más clara a medida que avancemos.

## Recall: Concatenación de matrices NumPy

La concatenación de objetos ``Series`` y ``DataFrame`` es muy similar a la concatenación de arrays Numpy, que se puede hacer a través de la función ``np.concatenate``.

Recuerda que con ella puedes combinar el contenido de dos o más arrays en un único array:

In [ ]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

El primer argumento es una lista o tupla de matrices a concatenar.

Además, toma una palabra clave ``axis`` que permite especificar el eje a lo largo del cual se concatenará el resultado:

In [ ]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

## Concatenación simple con ``pd.concat``

Pandas tiene una función, ``pd.concat()``, que tiene una sintaxis similar a ``np.concatenate`` pero contiene una serie de opciones que discutiremos momentáneamente:

```python
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

``pd.concat()`` puede utilizarse para una simple concatenación de objetos ``Series`` o ``DataFrame``, al igual que ``np.concatenate()`` puede utilizarse para simples concatenaciones de arrays:

In [ ]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

También funciona para concatenar objetos de mayor dimensión, como ``DataFrame``:

In [ ]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

Por defecto, la concatenación se realiza por filas dentro del ``DataFrame`` (es decir, ``axis=0``).
Al igual que ``np.concatenate``, ``pd.concat`` permite especificar un eje a lo largo del cual se producirá la concatenación.

Considere el siguiente ejemplo:

In [ ]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis='col')")

Podríamos haber especificado ``axis=1``; aquí hemos utilizado el más intuitivo ``axis='col'``.

### Índices duplicados

Una diferencia importante entre ``np.concatenate`` y ``pd.concat`` es que la concatenación en Pandas *preserva los índices*, ¡incluso si el resultado tiene índices duplicados!


In [ ]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # ¡hacer índices duplicados!
display('x', 'y', 'pd.concat([x, y])')

Observe los índices repetidos en el resultado.

Aunque esto es válido dentro de ``DataFrame``, el resultado es a menudo indeseable.

``pd.concat()`` nos da algunas maneras de manejarlo.

#### Captura de las repeticiones como error

Si quieres simplemente verificar que los índices en el resultado de ``pd.concat()`` no se solapan, puedes especificar la bandera ``verify_integrity``.

Con True, la concatenación lanzará una excepción si hay índices duplicados.

In [ ]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

#### Ignorar el índice

A veces, el índice en sí no importa y se prefiere ignorarlo.

Esta opción se puede especificar utilizando el indicador ``ignorar_índice``.

Si se establece en true, la concatenación creará un nuevo índice entero para la ``Series`` resultante:

In [ ]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

### Concatenación con uniones

En los ejemplos sencillos que acabamos de ver, concatenábamos principalmente ``DataFrame`` con nombres de columna compartidos.

En la práctica, los datos de diferentes fuentes pueden tener diferentes conjuntos de nombres de columna, y ``pd.concat`` ofrece varias opciones en este caso.

Considera la concatenación de los siguientes dos ``DataFrame``, que tienen algunas (¡pero no todas!) columnas en común:

In [ ]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

Por defecto, las entradas para las que no hay datos disponibles se rellenan con valores NA.

Para cambiar esto, podemos especificar una de varias opciones para los parámetros ``join`` y ``join_axes`` de la función concatenar.

Por defecto, la unión es una unión de las columnas de entrada (``join='outer'``), pero podemos cambiar esto a una intersección de las columnas usando ``join='inner'``:

In [ ]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

Otra opción es especificar directamente el índice de las columnas restantes utilizando el argumento ``reindex``, que toma una lista de objetos índice.

Aquí especificaremos que las columnas devueltas deben ser las mismas que las de la primera entrada:

In [ ]:
display('df5', 'df6',
        "pd.concat([df5, df6.reindex(columns=df5.columns)], axis=0, join='outer')")

La combinación de opciones de la función ``pd.concat`` permite una amplia gama de comportamientos posibles al unir dos ``DataFrame``.

En la siguiente sección, veremos otro enfoque más potente para combinar datos de múltiples fuentes, las fusiones/uniones al estilo de las bases de datos implementadas en ``pd.merge``.

<!--NAVIGATION-->
< [Valores Missing](4-Valores_missing.ipynb) | [Combinación de datos: Merge y Join](6-Merge_y_Join.ipynb) >

